In [1]:
# !pip install deck_of_cards 

In [2]:
from deck_of_cards import deck_of_cards
from scipy.special import comb


In [3]:
def play_game(hand_size=5):
    obj.shuffle_deck()
    d = [x.name for x in obj.deck]
    return d[:hand_size]

def nck(n,k):
    return comb(n,k,exact=True)
obj = deck_of_cards.DeckOfCards()

In [20]:
games=500000
win = 0
for x in range(games):

    hand = play_game()
    card_type = []
    for card in hand:
        card_type.append(card.split(" ")[0])
    four_of_a_kind=False
    for card in card_type:
        if card_type.count(card) == 4:
            four_of_a_kind = True
            break
    if(four_of_a_kind):
        win +=1
        print(hand)     
print(win/games)
win

['Queen of hearts', '9 of clubs', 'Queen of spades', 'Queen of diamonds', 'Queen of clubs']
['King of hearts', 'King of clubs', 'King of diamonds', 'Queen of spades', 'King of spades']
['8 of hearts', '8 of spades', '7 of clubs', '8 of clubs', '8 of diamonds']
['2 of spades', '2 of hearts', '2 of clubs', '3 of diamonds', '2 of diamonds']
['Queen of hearts', '8 of clubs', 'Queen of spades', 'Queen of diamonds', 'Queen of clubs']
['10 of spades', '10 of diamonds', '10 of clubs', '9 of clubs', '10 of hearts']
['9 of diamonds', '9 of hearts', '9 of spades', '9 of clubs', '8 of spades']
['2 of diamonds', '2 of spades', '6 of hearts', '2 of clubs', '2 of hearts']
['8 of spades', '8 of hearts', '8 of clubs', '7 of spades', '8 of diamonds']
['10 of hearts', '5 of clubs', '10 of diamonds', '10 of spades', '10 of clubs']
['8 of spades', '8 of hearts', '8 of diamonds', 'Queen of spades', '8 of clubs']
['5 of clubs', '5 of diamonds', '5 of spades', '5 of hearts', '10 of hearts']
['King of clubs', 

['4 of hearts', '4 of clubs', '5 of diamonds', '4 of diamonds', '4 of spades']
['King of clubs', 'Jack of spades', 'Jack of clubs', 'Jack of hearts', 'Jack of diamonds']
['7 of spades', '7 of clubs', '2 of hearts', '7 of hearts', '7 of diamonds']
['7 of hearts', '10 of spades', '10 of clubs', '10 of diamonds', '10 of hearts']
['4 of diamonds', '4 of clubs', 'Jack of diamonds', '4 of spades', '4 of hearts']
['3 of diamonds', '3 of clubs', '3 of spades', '4 of clubs', '3 of hearts']
0.000216


108

In [19]:
nck(52,5)

2598960

In [ ]:
[x.name for x in deck.deck]

In [ ]:
deck.shuffle_deck()